In [57]:
import pandas as pd
import numpy as np
import os
import json

oscwd = os.getcwd()

"codigo_entidad","fecha_informacion","situacion","tramo","desc_tramo","total"
# Load tramos de préstamos
df_tra_pres = pd.read_csv(os.path.join(oscwd, "data/dbo_Men_TraPres.txt"),
                            dtype = {
                                    "codigo_entidad": str, "fecha_informacion": str, 
                                    "situacion": str, "tramo": int,
                                    "desc_tramo": str, "total": float
                                    }
                                    # parse_dates=['Periodo'],
                            )

df_tra_pres.columns = ["bank", "per", "sit", "seg", "seg_descrip", "total"]
# Load IPC
df_ipc = pd.read_csv(os.path.join(oscwd, "data/IPC.txt"),
                            dtype = {
                                    "per": str, "ipc": float
                                    }
                                    # parse_dates=['Periodo'],
                            )

In [58]:
import re
df_tra_pres["threshold"] = [t.replace(".", "") for t in df_tra_pres["seg_descrip"]]
df_tra_pres["threshold"] = [t.replace(",", ".") for t in df_tra_pres["threshold"]]
ratio_list = [re.findall(r"[-+]?(?:\d*\.\d+|\d+)", t) for t in df_tra_pres["threshold"]]
tra_list_upper = list(set([float(r[e]) for r in ratio_list for e in range(len(r))]))
tra_list_upper.insert(0, 100000.0)
tra_list_upper.sort(reverse=False)
# display(tra_list_upper)
# print(len(tra_list_upper))

In [59]:
tra_list_lower = list(set([float(r[e]) for r in ratio_list for e in range(len(r))]))
tra_list_lower.insert(0,0.0)
tra_list_lower.sort(reverse=False)
# print(tra_list_lower)
# len(tra_list_lower)

In [62]:
tra_list = list(set([int(t) for t in df_tra_pres["seg"]]))
# tra_list.sort(reverse=False)
# print(tra_list)
# print(len(tra_list))

In [64]:
df_tra_avg = pd.DataFrame({"n_seg": tra_list, "lower_b": tra_list_lower, "upper_b": tra_list_upper})
df_tra_avg["avg_thr"] = ((df_tra_avg["upper_b"] + df_tra_avg["lower_b"]) / 2) * 1
#df_tra_avg

In [66]:
# df_tra_pres["ent_per_tra"] = df_tra_pres["codigo_entidad"]+"_"+df_tra_pres["fecha_informacion"]+"_"+df_tra_pres["tramo"]
df_tra_pres_agg = df_tra_pres.groupby(["bank", "per", "seg"]).mean("total").reset_index()
df_tra_pres_agg["avg"] = df_tra_pres_agg["total"]
df_tra_pres_agg["avg_total"] = df_tra_pres_agg.groupby(["bank", "per"])["avg"].transform("sum")
df_tra_pres_agg = df_tra_pres_agg.drop(columns=["total"])
df_tra_pres_agg["avg_threshold"] = [df_tra_avg["avg_thr"].loc[df_tra_avg.index[df_tra_avg["n_seg"]==t][0]] for t in df_tra_pres_agg["seg"]]
df_tra_pres_agg["share"] = df_tra_pres_agg["avg"] / df_tra_pres_agg["avg_total"]
df_tra_pres_agg["ipc"] = [df_ipc["ipc"].loc[df_ipc.index[df_ipc["per"]==p][0]] for p in df_tra_pres_agg["per"]]
df_tra_pres_agg["weighted_value"] = (df_tra_pres_agg["avg_threshold"] * df_tra_pres_agg["share"]) * (1 + df_tra_pres_agg["ipc"])
df_tra_pres_agg["acc"] = "10399999999"
df_tra_pres_agg[df_tra_pres_agg["bank"]=="00011"].sort_values("seg")

,bank,per,seg,avg,avg_total,avg_threshold,share,ipc,weighted_value,acc
20,00011,2015,1,6.139053e+05,1.672125e+10,0.15,0.000037,0.254,0.000007,10399999999
21,00011,2015,2,8.832311e+05,1.672125e+10,0.40,0.000053,0.254,0.000026,10399999999
22,00011,2015,3,1.208151e+07,1.672125e+10,0.75,0.000723,0.254,0.000680,10399999999
23,00011,2015,4,6.718962e+07,1.672125e+10,1.75,0.004018,0.254,0.008818,10399999999
24,00011,2015,5,1.223298e+08,1.672125e+10,3.75,0.007316,0.254,0.034403,10399999999
25,00011,2015,6,2.604769e+08,1.672125e+10,7.50,0.015578,0.254,0.146507,10399999999
26,00011,2015,7,5.598319e+08,1.672125e+10,17.50,0.033480,0.254,0.734725,10399999999
27,00011,2015,8,5.429394e+08,1.672125e+10,37.50,0.032470,0.254,1.526903,10399999999
28,00011,2015,9,7.632088e+08,1.672125e+10,75.00,0.045643,0.254,4.292730,10399999999
29,00011,2015,10,8.538506e+08,1.672125e+10,150.00,0.051064,0.254,9.605103,10399999999


In [81]:
df_tra_wv = df_tra_pres_agg.groupby(["bank", "per", "acc"]).sum("weighted_value").reset_index()
df_tra_wv = df_tra_wv[["bank", "per", "acc", "weighted_value"]]
df_tra_wv
df_tra_wv.to_csv("data/df_loan_seg_avg_wv.csv", sep=",", index=False)